In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import itertools
import json
from pathlib import Path
from datetime import timedelta, datetime

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from wikipedia_cleanup.data_processing import read_file, get_data
from wikipedia_cleanup.baseline import next_change

In [ ]:
mp_drive_dir = Path("/media/hpi_share/")#Path("//FS23/projekte$/MP2021/MPWS2021/MPWS2021FN1")
mp_plot_dir = mp_drive_dir / "plots"
input_path = Path("../../data")
input_data = list(input_path.rglob("*.json"))
files = [x for x in input_data if x.is_file()]
len(files) # total 580

In [ ]:
data = get_data(input_path, n_files=3, n_jobs=0)
data['validFrom'] = data['validFrom'].dt.tz_localize(None)
data['validTo'] =  data['validTo'].dt.tz_localize(None)

In [ ]:
timestamps_per_page_property = train_data.groupby(['pageID', 'property.name'])['validFrom'].count()

In [ ]:
testset_start = datetime(2018,9,1)
testset_duration = 365
total_time_window = timedelta(testset_duration)#days
testset_end = testset_start + total_time_window
time_offset = timedelta(1)

In [ ]:
np.searchsorted([1,2,3,4,4,4,5], 4, side="right")

In [ ]:
property_change_history = data.groupby(['pageID', 'property.name'])["validFrom"].agg(list)

In [ ]:
class DummyPredictor:
    def predict_day(data, current_day):
        return np.random.randint(0,2)
    def predict_week(data, current_day):
        return np.random.randint(0,2)
    def predict_month(data, current_day):
        return np.random.randint(0,2)
    def predict_year(data, current_day):
        return np.random.randint(0,2)

In [ ]:
changes[0].date() == current_date.date()

In [ ]:
all_year_predictions = []
all_month_predictions = []
all_week_predictions = []
all_day_predictions = []
all_day_labels = []
model = DummyPredictor()
for key, changes in tqdm(property_change_history.iteritems(), total=len(property_change_history)):
    days_evaluated = 0
    current_date = testset_start
    changes = sorted(changes)
    train_data_idx = np.searchsorted(changes, current_date, side="right")
    day_predictions = np.empty(testset_duration)
    week_predictions = []
    month_predictions = []
    year_predictions = []
    day_labels = []
    while current_date < testset_end:        
        
        #day_predictions.append(model.predict_day(changes[:train_data_idx]))
        #if days_evaluated % 7 == 0:
        #     week_predictions.append(model.predict_week(changes[:train_data_idx]))
        #if days_evaluated % 30 == 0:
        #    month_predictions.append(model.predict_month(changes[:train_data_idx]))
        #if days_evaluated % 365 == 0:
        #    year_predictions.append(model.predict_year(changes[:train_data_idx]))
        #day_labels.append(changes[train_data_idx-1].date() == current_date.date())
        days_evaluated += 1
        current_date += time_offset
        while(train_data_idx < len(changes) and changes[train_data_idx] < current_date):
            train_data_idx+=1
    all_day_predictions.append(day_predictions)
    all_week_predictions.append(week_predictions)
    all_month_predictions.append(month_predictions)
    all_year_predictions.append(year_predictions)

In [ ]:
all_year_predictions = np.array(all_year_predictions)
all_month_predictions = np.array(all_month_predictions)
all_week_predictions = np.array(all_week_predictions)
all_day_predictions = np.array(all_day_predictions)

In [ ]:
# cut off general time frame
train_data = data[data["validFrom"] < testset_start]
test_data = data[(testset_start < data["validFrom"]) & (data["validFrom"] < testset_end)]

In [ ]:
timestamps_per_page_property